In [2]:
# MNIST dataset
import torchvision
import torch
import torchvision.transforms as transforms



In [52]:
import torch.nn as nn
torch.manual_seed(42)


In [ ]:

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # 1 x 28 x 28
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2), # 32 x 28 x 28
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))  # 32 x 14 x 14
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), # 64 x 14 x 14
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)) # 64 x 7 x 7
        self.drop_out = torch.nn.Dropout()
        self.fc1 = torch.nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = torch.nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [69]:
dataset_train[0][0].shape

torch.Size([1, 28, 28])

In [84]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        # 1 x 28 x 28
        self.convs = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=3,padding=1), nn.ReLU(), # 96 x 28 x 28
            nn.MaxPool2d(kernel_size=2, stride=2), # 96 x 14 x 14
            nn.Conv2d(96, 256, kernel_size=3, padding=1), nn.ReLU(), # 256 x 14 x 14
            nn.MaxPool2d(kernel_size=2, stride=2), # 256 x 7 x 7
            nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(), # 384 x 7 x 7
            nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(), # 256 x 7 x 7
            nn.MaxPool2d(kernel_size=2, stride=2), # 256 x 3 x 3
            nn.Flatten())
        self.model = nn.Sequential(
            nn.Linear(256 * 3 * 3, 4096), nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096), nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 10))
    def forward(self, x):
        x = self.convs(x)
        return self.model(x)
    

In [76]:
dataset_test = torchvision.datasets.MNIST(root='./datasets', train=False, transform=transforms.ToTensor(), download=True)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=10000, shuffle=False)

dataset_train = torchvision.datasets.MNIST(root='./datasets', train=True, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=100, shuffle=True)

In [77]:
class Config:
    lr = 0.001
    
   

In [78]:
config = Config()

In [79]:
import lightning as L

class LightningNN(L.LightningModule):
    def __init__(self, model, config):
        super().__init__()
        self.model = model
        self.loss = torch.nn.CrossEntropyLoss()
        self.config = config
    
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        
        return torch.optim.Adam(self.model.parameters(), lr=self.config.lr)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = self.loss(y_hat, y)
        return loss
    
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self.model(x)
        y_pred = torch.argmax(y_hat, dim=1)
        return y_pred
    


In [85]:
# model = AlexNet()
model = ConvNet()

In [86]:
lightning_model = LightningNN(model, config)


In [87]:
trainer = L.Trainer(limit_train_batches=100, max_epochs=10)
trainer.fit(model=lightning_model, train_dataloaders=train_loader)

Epoch 0:   0%|          | 0/100 [19:50<?, ?it/s]

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type             | Params
-------------------------------------------
0 | model | AlexNet          | 29.6 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
29.6 M    Trainable params
0         Non-trainable params
29.6 M    Total params
118.334   Total estimated model params size (MB)



Epoch 9: 100%|██████████| 100/100 [00:49<00:00,  2.03it/s, loss=0.0406, v_num=11]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 100/100 [00:50<00:00,  2.00it/s, loss=0.0406, v_num=11]


In [50]:
def test(trainer, lightning_model, test_loader):
    y_pred = trainer.predict(model=lightning_model, dataloaders=test_loader)[0]
    y_gt = torch.cat([y for x, y in test_loader], dim=0)
    acc = (y_pred == y_gt).sum().item() / y_gt.size(0)
    return acc

In [88]:
test(trainer, lightning_model, test_loader)

c:\Users\suzy\miniconda3\envs\abbasi\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:229: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 100it [00:00, ?it/s]

Predicting DataLoader 0: 100%|██████████| 1/1 [00:09<00:00,  9.18s/it]


0.9874

In [ ]:
## An arbitrary convnet
## Accuracy 0.9894

## AlexNet
## Accuracy 0.9874